# mt5-thai-QG
This is notebook detailing how to finetune **mt5 for question-generation in the Thai language** 
## Installing Requirements and Clone Git Repo

In [1]:
# Remove "sample_data" in colab
!rm -rf sample_data
# Solve some protobuf problems
!export PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION='python'
# Install programs
%pip install ijson pandas torch transformers sentencepiece protobuf beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


## Gather & Process datasets
- xquad-thai
- iapp-wiki-qa-dataset
- thaiqa

In [6]:

import math
import pandas as pd
import urllib.request
import os
import ijson
import json
import re
from zipfile import ZipFile
from bs4 import BeautifulSoup


def download_dataset(url, file_name):
    urllib.request.urlretrieve(
        url,
        os.path.join("dataset/", file_name),
        reporthook=(
            lambda count, block, total: print(
                f"Downloading {file_name}: {math.floor((count * block) / total * 100)}%",
                end="\r",
            )
        ),
    )
    print(f"Downloaded {file_name} from {url}")


# Check if the dataset already exists
if not (os.path.exists("dataset/xquad.json") and os.path.exists("dataset/iapp-thai-wikipedia-qa.json")):

    # Download all datasets
    download_dataset("https://github.com/deepmind/xquad/raw/master/xquad.th.json", "xquad.json")
    download_dataset("https://raw.githubusercontent.com/iapp-technology/iapp-wiki-qa-dataset/main/iapp-thai-wikipedia-qa-1961-docs-9170-questions.json", "iapp-thai-wikipedia-qa.json")
    download_dataset("https://github.com/PyThaiNLP/thaiqa_squad/raw/main/data.zip", "thaiqa.zip")
    with ZipFile("dataset/thaiqa.zip") as zipfile:
        os.mkdir("dataset/thaiqa")
        zipfile.extractall("dataset/thaiqa/")

# This list will store all the Q&A
source_list = []
target_list = []

# Start cleaning data
squad = open(os.path.join("dataset/", "xquad.json"))
iapp = open(os.path.join("dataset/", "iapp-thai-wikipedia-qa.json"))
iapp_keys = open(os.path.join("dataset/", "iapp-thai-wikipedia-qa.json"))
thaiqa = open(os.path.join("dataset/thaiqa/data/train.jsonl"))

squad_json = ijson.items(squad, "data.item")
iapp_json = json.load(iapp)
iapp_keys = ijson.kvitems(iapp_keys, "db")
thaiqa_df = pd.read_json(thaiqa, lines=True)

# Get data from xquad
for obj in squad_json:
    paragraphs = obj["paragraphs"]
    for p in paragraphs:
        context = p["context"]
        qas = [p for p in p["qas"] if len(p) > 0]

        source_text = f"สร้าง {len(qas)} คำถาม: {context}"
        target_text = ""

        for number, qa in enumerate(qas):
            target_text += (
                f"{number + 1}. {qa['question']} A: {qa['answers'][0]['text']} "
            )

        source_list.append(source_text.strip())
        target_list.append(target_text.strip())

# Get dataset from iapp
for key in iapp_keys:
    try:
        obj = iapp_json["db"][key[0]] 
        context = obj["detail"]
        qas = obj["QA"]
        target_text = ""

        qa_amount = 0

        for number, qa in enumerate(qas):
            if len(qa["a"]) != 0 and len(qa["q"]) != 0:
                target_text += (
                    f"{number + 1}. {qa['q']} A: {qa['a'][0]} "
                )
                qa_amount += 1

        source_text = f"สร้าง {qa_amount} คำถาม: {context}"
        source_list.append(source_text.strip())
        target_list.append(target_text.strip())
    
    except KeyError as e:
        print(f"Error: {e} (Error on 'detail' is expected because of the dataset structure")
    
# Get data from thaiqa
article_ids = set(thaiqa_df["article_id"])
for id in article_ids:
    questions = thaiqa_df[thaiqa_df["article_id"] == id]

    # Remove html markup
    soup = BeautifulSoup(questions["context"].iloc[0])

    # Remove parenthesis because some are empty
    context = re.sub(r'\(\)', '', soup.text)

    # Remove double spaces resulting from removing parenthesis
    context = re.sub(r'\s\s+', " ", context)

    source_text = f"สร้าง {len(questions)} คำถาม: {context}"
    target_text = ""

    qa_number = 1
    for _, question in questions.iterrows():
        target_text += f"{qa_number}. {question['question']} A: {question['answer']} "
        qa_number += 1
    
    source_list.append(source_text.strip())
    target_list.append(target_text.strip())

dataframe = pd.DataFrame({"source_text": source_list, "target_text": target_list})


Error: 'detail' (Error on 'detail' is expected because of the dataset structure


,source_text,target_text
1725,สร้าง 5 คำถาม: ชื่อสินค้า กระจก Washi Haruki W...,1. ขอทราบชื่อสินค้าหน่อย A: กระจก Washi Haruki...
1570,สร้าง 4 คำถาม: ภาษาโปรแกรม คือภาษาประดิษฐ์ชนิด...,1. ภาษาโปรแกรมคืออะไร A: ภาษาประดิษฐ์ชนิดหนึ่ง...
1483,สร้าง 5 คำถาม: แม่น้ำลิมโปโป (อังกฤษ: Limpopo ...,1. แม่น้ำลิมโปโป เป็นแม่น้ำในทวีปอะไร A: ทวีปแ...
680,สร้าง 1 คำถาม: อะกลีอา (อังกฤษ: Aglaea) หรือ อ...,1. อะกลีอาคือใคร A: เทพีองค์เล็กที่สุดในสามองค...
2495,สร้าง 2 คำถาม: หวง เสี่ยวหมิง หวง เสี่ยวหมิง (...,1. หวง เสี่ยวหมิง เป็นนักแสดงและนักร้องชาวจีนท...
...,...,...
1009,สร้าง 4 คำถาม: เดทเมทัล (อังกฤษ: Death metal) ...,1. เดทเมทัลคืออะไร A: แนวเพลงย่อยของเอกซ์ตรีมเ...
2649,สร้าง 2 คำถาม: ภัสสร บุณยเกียรติ ภัสสร เหลียวร...,1. ภัสสร บุณยเกียรติได้รับรางวัลใดจากการประกวด...
3198,สร้าง 2 คำถาม: การเต้นแท็ป การเต้นแท็ป หรือ แท...,1. การเต้นแท็ปมีต้นกำเนิดมาจากประเทศอะไร A: สห...
1048,"สร้าง 3 คำถาม: โรเบิร์ต ไมเคิล ""ร็อบ"" ชไนเดอร์...",1. ร็อบ ชไนเดอร์ เกิดเมื่อวันที่เท่าไร A: เกิด...


## Training

In [22]:
from transformers import (
    MT5ForConditionalGeneration,
    MT5TokenizerFast,
    DataCollatorForSeq2Seq,
    TrainingArguments,
    Trainer
)
import numpy as np

model = MT5ForConditionalGeneration.from_pretrained("google/mt5-small")
tokenizer = MT5TokenizerFast.from_pretrained("google/mt5-small")
collator = DataCollatorForSeq2Seq(tokenizer, model)

tokenized_data = []

# Apply tokenizer
for _, row in dataframe.iterrows():
    source = row["source_text"]
    target = row["target_text"]
    max_length = 2048

    source_encoding = tokenizer(
        source,
        max_length=max_length,
        padding="max_length",
        truncation=True,
        return_attention_mask=True,
        return_special_tokens_mask=True,
        return_tensors="pt",
    )

    target_encoding = tokenizer(
        target,
        max_length=max_length,
        padding="max_length",
        truncation=True,
        return_attention_mask=True,
        return_special_tokens_mask=True,
        return_tensors="pt",
        )

    tokenized_data.append(
        dict(
            input_ids=source_encoding["input_ids"].flatten(),
            attention_mask=source_encoding["attention_mask"].flatten(),
            labels=target_encoding["input_ids"].flatten(),
            labels_attention_mask=target_encoding["attention_mask"].flatten(),
        )
    )

tokenized_data = np.array(tokenized_data)
train, test = tokenized_data[:80,:], tokenized_data[80:,:]

/home/parinzee/anaconda3/lib/python3.9/site-packages/transformers/convert_slow_tokenizer.py:434: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [34]:
import numpy as np
import math

tokenized_data = np.array(tokenized_data)
index_to_split = math.floor(tokenized_data.shape[0] * 0.8)
train, test = tokenized_data[:index_to_split], tokenized_data[index_to_split:]

(3592,)

In [ ]:
training_args = TrainingArguments("outputs", evaluation_strategy="epoch")
trainer = Trainer(model=model, args=TrainingArguments, data_collator=collator)
trainer.train()

In [ ]:
from simplet5 import SimpleT5

model = SimpleT5()
model.from_pretrained(model_type="mt5", model_name="google/mt5-small")

model.train(train_df=train_df,
          eval_df=test_df, 
          source_max_token_len=1024, 
          target_max_token_len=1024, 
          batch_size=1, max_epochs=20, use_gpu=True)

## Load trained model

In [ ]:
# let's load the trained model for inferencing:
model.load_model("mt5","", use_gpu=True)

text_to_summarize="""สร้าง 3 คำถาม: ดันเจียนซีจ (อังกฤษ: Dungeon Siege) เป็นเกมแอ็กชันเล่นตามบทบาทที่พัฒนาโดยแก๊สเพาเวิร์ดเกมส์ ซึ่งไมโครซอฟท์ได้จัดจำหน่ายบนแพลตฟอร์มไมโครซอฟท์ วินโดวส์ ในเดือนเมษายน ค.ศ. 2002 และเดสทิเนียร์ได้จัดจำหน่ายบนแพลตฟอร์มแมคโอเอสเท็นในปีถัดไป โดยมีฉากอยู่ในอาณาจักรยุคกลางสมมติ ชื่อ เอห์บ เกมนี้ยังจัดเป็นแนวแฟนตาซีระดับสูงที่เดินเรื่องตามชาวไร่หนุ่มคนหนึ่งและเพื่อนร่วมทางขณะที่พวกเขาออกเดินทางเพื่อกำจัดกองกำลังที่รุกราน ในตอนแรก กลุ่มตัวเอกเพียงต้องการเตือนเมืองใกล้เคียงเกี่ยวกับการรุกรานของเผ่าพันธุ์สิ่งมีชีวิตที่ชื่อครุก และในอีกไม่นาน ชาวไร่คนดังกล่าวและเพื่อนร่วมทางกับเขาตกอยู่ในสถานการณ์หาทางเอาชนะเผ่าพันธุ์อื่นที่เรียกว่าเซกอย่างหลีกเลี่ยงไม่ได้ ซึ่งฟื้นคืนพลังใหม่หลังจากถูกคุมขังอยู่ 300 ปี โลกของดันเจียนซีจไม่ใช้ระบบเลเวลเหมือนกับวิดีโอเกมเล่นตามบทบาทอื่น ๆ ในยุคนั้น หากแต่เป็นพื้นที่เดียวที่ต่อเนื่อง โดยปราศจากการโหลดหน้าจอ ซึ่งผู้เล่นเดินทางผ่านเพื่อต่อสู้กับฝูงศัตรู นอกจากนี้ แทนที่จะกำหนดคลาสตัวละครและควบคุมตัวละครทั้งหมดในกลุ่มด้วยตนเอง ผู้เล่นจะควบคุมกลยุทธ์และอาวุธ ตลอดจนการใช้เวทมนตร์โดยรวมของพวกเขา ซึ่งกำกับการเติบโตของตัวละคร
"""

print(model.predict(text_to_summarize, max_length=1024)[0])